# Dataset Cleaning 

The main objectives of this notebook are: 
- Import dataset 
- Get familiar with the data 
- Determine if there are data quality issues 
- Resolve any data quality issues 

In [33]:
import polars as pl 
import polars.selectors as cs 
from google.cloud import bigquery

In [34]:
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [35]:
# Create client object
client = bigquery.Client()

# Create dataset reference
dataset_ref = client.dataset('google_analytics_sample', project='bigquery-public-data')

# Retrieve dataset from reference
dataset = client.get_dataset(dataset_ref)

In [36]:
QUERY = ("SELECT `date`, `visitId`,`visitNumber`,`visitStartTime`,`visitorId`,`userId`,"
         "`trafficSource`.`referralPath`,`trafficSource`.`source`,`device`.`browser`,"
         "`device`.`operatingSystem`,`device`.`isMobile`,`device`.`deviceCategory`,"
         "`geoNetwork`.`continent`,`geoNetwork`.`subContinent`,`geoNetwork`.`country`,"
         "`geoNetwork`.`region`,`geoNetwork`.`metro`,`geoNetwork`.`city`,`totals`.`visits`,"
         "`totals`.`pageviews`,`totals`.`timeOnSite`,`totals`.`transactions`,`totals`.`transactionRevenue`,"
         "`totals`.`totalTransactionRevenue`,"
         "FROM`bigquery-public-data.google_analytics_sample.ga_sessions_20170801`;")

In [37]:
query_job = client.query(QUERY)

In [38]:
df = pl.from_arrow(query_job.to_arrow())

c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [39]:
df.head()

date,visitId,visitNumber,visitStartTime,visitorId,userId,referralPath,source,browser,operatingSystem,isMobile,deviceCategory,continent,subContinent,country,region,metro,city,visits,pageviews,timeOnSite,transactions,transactionRevenue,totalTransactionRevenue
str,i64,i64,i64,i64,str,str,str,str,str,bool,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64
"""20170801""",1501591568,1,1501591568,null,null,null,"""(direct)""","""Chrome""","""Windows""",false,"""desktop""","""Europe""","""Southern Europe""","""Greece""","""not available in demo dataset""","""not available in demo dataset""","""not available in demo dataset""",1,1,null,null,null,null
"""20170801""",1501589647,2,1501589647,null,null,"""/analytics/web/""","""analytics.google.com""","""Chrome""","""Windows""",false,"""desktop""","""Asia""","""Southern Asia""","""India""","""Maharashtra""","""(not set)""","""Mumbai""",1,1,null,null,null,null
"""20170801""",1501616621,1,1501616621,null,null,"""/analytics/web/""","""analytics.google.com""","""Chrome""","""Windows""",false,"""desktop""","""Europe""","""Northern Europe""","""United Kingdom""","""not available in demo dataset""","""not available in demo dataset""","""not available in demo dataset""",1,1,null,null,null,null
"""20170801""",1501601200,1,1501601200,null,null,"""/analytics/web/""","""analytics.google.com""","""Firefox""","""Windows""",false,"""desktop""","""Americas""","""Northern America""","""United States""","""Texas""","""Dallas-Ft. Worth TX""","""Dallas""",1,1,null,null,null,null
"""20170801""",1501615525,1,1501615525,null,null,"""/analytics/web/""","""adwords.google.com""","""Chrome""","""Windows""",false,"""desktop""","""Americas""","""Northern America""","""United States""","""not available in demo dataset""","""not available in demo dataset""","""not available in demo dataset""",1,1,null,null,null,null


In [40]:
df.shape

(2556, 24)

In [41]:
df.columns

['date',
 'visitId',
 'visitNumber',
 'visitStartTime',
 'visitorId',
 'userId',
 'referralPath',
 'source',
 'browser',
 'operatingSystem',
 'isMobile',
 'deviceCategory',
 'continent',
 'subContinent',
 'country',
 'region',
 'metro',
 'city',
 'visits',
 'pageviews',
 'timeOnSite',
 'transactions',
 'transactionRevenue',
 'totalTransactionRevenue']

In [42]:
df.sample()

date,visitId,visitNumber,visitStartTime,visitorId,userId,referralPath,source,browser,operatingSystem,isMobile,deviceCategory,continent,subContinent,country,region,metro,city,visits,pageviews,timeOnSite,transactions,transactionRevenue,totalTransactionRevenue
str,i64,i64,i64,i64,str,str,str,str,str,bool,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64
"""20170801""",1501600013,1,1501600013,null,null,null,"""(direct)""","""Internet Explorer""","""Windows""",false,"""desktop""","""Europe""","""Western Europe""","""Germany""","""not available in demo dataset""","""not available in demo dataset""","""not available in demo dataset""",1,1,null,null,null,null


In [43]:
df.describe()

statistic,date,visitId,visitNumber,visitStartTime,visitorId,userId,referralPath,source,browser,operatingSystem,isMobile,deviceCategory,continent,subContinent,country,region,metro,city,visits,pageviews,timeOnSite,transactions,transactionRevenue,totalTransactionRevenue
str,str,f64,f64,f64,f64,str,str,str,str,str,f64,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64
"""count""","""2556""",2556.0,2556.0,2556.0,0.0,"""0""","""649""","""2556""","""2556""","""2556""",2556.0,"""2556""","""2556""","""2556""","""2556""","""2556""","""2556""","""2556""",2556.0,2556.0,1310.0,43.0,43.0,43.0
"""null_count""","""0""",0.0,0.0,0.0,2556.0,"""2556""","""1907""","""0""","""0""","""0""",0.0,"""0""","""0""","""0""","""0""","""0""","""0""","""0""",0.0,0.0,1246.0,2513.0,2513.0,2513.0
"""mean""",null,1.5016e9,2.661189,1.5016e9,null,null,null,null,null,null,0.318858,null,null,null,null,null,null,null,1.0,4.279734,325.051908,1.046512,1.9314e8,2.0674e8
"""std""",null,22804.553787,12.0222,22804.234297,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,7.966343,592.411724,0.213083,4.7870e8,4.7852e8
"""min""","""20170801""",1.5016e9,1.0,1.5016e9,null,null,"""/""","""(direct)""","""Android Browser""","""(not set)""",0.0,"""desktop""","""(not set)""","""(not set)""","""(not set)""","""(not set)""","""(not set)""","""(not set)""",1.0,1.0,1.0,1.0,1.99e6,2.99e6
"""25%""",null,1.5016e9,1.0,1.5016e9,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0,1.0,32.0,1.0,2.958e7,3.479e7
"""50%""",null,1.5016e9,1.0,1.5016e9,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0,2.0,97.0,1.0,4.567e7,5.079e7
"""75%""",null,1.5016e9,2.0,1.5016e9,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0,4.0,337.0,1.0,1.5714e8,1.739e8
"""max""","""20170801""",1.5017e9,326.0,1.5017e9,null,null,"""/yt/about/press/""","""youtube.com""","""YaBrowser""","""iOS""",1.0,"""tablet""","""Oceania""","""Western Europe""","""Vietnam""","""not available in demo dataset""","""not available in demo dataset""","""not available in demo dataset""",1.0,155.0,5455.0,2.0,2.9336e9,2.9356e9


In [44]:
df.filter(pl.col('transactionRevenue') > 0)

date,visitId,visitNumber,visitStartTime,visitorId,userId,referralPath,source,browser,operatingSystem,isMobile,deviceCategory,continent,subContinent,country,region,metro,city,visits,pageviews,timeOnSite,transactions,transactionRevenue,totalTransactionRevenue
str,i64,i64,i64,i64,str,str,str,str,str,bool,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64
"""20170801""",1501621191,4,1501621191,null,null,null,"""(direct)""","""Chrome""","""Macintosh""",false,"""desktop""","""Americas""","""Northern America""","""United States""","""California""","""San Francisco-Oakland-San Jose…","""Mountain View""",1,5,56,1,35290000,40290000
"""20170801""",1501612542,6,1501612542,null,null,"""/""","""(direct)""","""Chrome""","""Macintosh""",false,"""desktop""","""Americas""","""Northern America""","""United States""","""California""","""San Francisco-Oakland-San Jose…","""Mountain View""",1,11,212,1,27430000,30430000
"""20170801""",1501599317,1,1501599317,null,null,null,"""(direct)""","""Chrome""","""Macintosh""",false,"""desktop""","""Americas""","""Northern America""","""United States""","""not available in demo dataset""","""not available in demo dataset""","""not available in demo dataset""",1,14,559,1,1990000,2990000
"""20170801""",1501620679,3,1501620679,null,null,null,"""(direct)""","""Safari""","""Macintosh""",false,"""desktop""","""Americas""","""Northern America""","""United States""","""not available in demo dataset""","""not available in demo dataset""","""not available in demo dataset""",1,14,237,1,13290000,22290000
"""20170801""",1501595478,3,1501595478,null,null,"""/mail/ca/u/0/""","""mail.google.com""","""Chrome""","""Macintosh""",false,"""desktop""","""Americas""","""Northern America""","""United States""","""New York""","""New York NY""","""New York""",1,16,1474,1,11960000,17960000
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""20170801""",1501628356,8,1501628356,null,null,"""/""","""(direct)""","""Chrome""","""Macintosh""",false,"""desktop""","""Americas""","""Northern America""","""United States""","""Illinois""","""Chicago IL""","""Chicago""",1,48,1827,2,126680000,400960000
"""20170801""",1501647149,1,1501647149,null,null,null,"""(direct)""","""Chrome""","""Windows""",false,"""desktop""","""Americas""","""Northern America""","""United States""","""not available in demo dataset""","""not available in demo dataset""","""not available in demo dataset""",1,58,1685,1,4990000,6990000
"""20170801""",1501608078,8,1501608078,null,null,"""/""","""(direct)""","""Chrome""","""Macintosh""",false,"""desktop""","""Americas""","""Northern America""","""United States""","""California""","""San Francisco-Oakland-San Jose…","""San Francisco""",1,55,1572,1,2933610000,2935610000


In [45]:
df.select(pl.all().is_null().sum())

date,visitId,visitNumber,visitStartTime,visitorId,userId,referralPath,source,browser,operatingSystem,isMobile,deviceCategory,continent,subContinent,country,region,metro,city,visits,pageviews,timeOnSite,transactions,transactionRevenue,totalTransactionRevenue
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,2556,2556,1907,0,0,0,0,0,0,0,0,0,0,0,0,0,1246,2513,2513,2513


In [46]:
df.head()

date,visitId,visitNumber,visitStartTime,visitorId,userId,referralPath,source,browser,operatingSystem,isMobile,deviceCategory,continent,subContinent,country,region,metro,city,visits,pageviews,timeOnSite,transactions,transactionRevenue,totalTransactionRevenue
str,i64,i64,i64,i64,str,str,str,str,str,bool,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64
"""20170801""",1501591568,1,1501591568,null,null,null,"""(direct)""","""Chrome""","""Windows""",false,"""desktop""","""Europe""","""Southern Europe""","""Greece""","""not available in demo dataset""","""not available in demo dataset""","""not available in demo dataset""",1,1,null,null,null,null
"""20170801""",1501589647,2,1501589647,null,null,"""/analytics/web/""","""analytics.google.com""","""Chrome""","""Windows""",false,"""desktop""","""Asia""","""Southern Asia""","""India""","""Maharashtra""","""(not set)""","""Mumbai""",1,1,null,null,null,null
"""20170801""",1501616621,1,1501616621,null,null,"""/analytics/web/""","""analytics.google.com""","""Chrome""","""Windows""",false,"""desktop""","""Europe""","""Northern Europe""","""United Kingdom""","""not available in demo dataset""","""not available in demo dataset""","""not available in demo dataset""",1,1,null,null,null,null
"""20170801""",1501601200,1,1501601200,null,null,"""/analytics/web/""","""analytics.google.com""","""Firefox""","""Windows""",false,"""desktop""","""Americas""","""Northern America""","""United States""","""Texas""","""Dallas-Ft. Worth TX""","""Dallas""",1,1,null,null,null,null
"""20170801""",1501615525,1,1501615525,null,null,"""/analytics/web/""","""adwords.google.com""","""Chrome""","""Windows""",false,"""desktop""","""Americas""","""Northern America""","""United States""","""not available in demo dataset""","""not available in demo dataset""","""not available in demo dataset""",1,1,null,null,null,null


In [47]:
df = df.drop(pl.col('visitorId','userId'))
df.head()

date,visitId,visitNumber,visitStartTime,referralPath,source,browser,operatingSystem,isMobile,deviceCategory,continent,subContinent,country,region,metro,city,visits,pageviews,timeOnSite,transactions,transactionRevenue,totalTransactionRevenue
str,i64,i64,i64,str,str,str,str,bool,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64
"""20170801""",1501591568,1,1501591568,null,"""(direct)""","""Chrome""","""Windows""",false,"""desktop""","""Europe""","""Southern Europe""","""Greece""","""not available in demo dataset""","""not available in demo dataset""","""not available in demo dataset""",1,1,null,null,null,null
"""20170801""",1501589647,2,1501589647,"""/analytics/web/""","""analytics.google.com""","""Chrome""","""Windows""",false,"""desktop""","""Asia""","""Southern Asia""","""India""","""Maharashtra""","""(not set)""","""Mumbai""",1,1,null,null,null,null
"""20170801""",1501616621,1,1501616621,"""/analytics/web/""","""analytics.google.com""","""Chrome""","""Windows""",false,"""desktop""","""Europe""","""Northern Europe""","""United Kingdom""","""not available in demo dataset""","""not available in demo dataset""","""not available in demo dataset""",1,1,null,null,null,null
"""20170801""",1501601200,1,1501601200,"""/analytics/web/""","""analytics.google.com""","""Firefox""","""Windows""",false,"""desktop""","""Americas""","""Northern America""","""United States""","""Texas""","""Dallas-Ft. Worth TX""","""Dallas""",1,1,null,null,null,null
"""20170801""",1501615525,1,1501615525,"""/analytics/web/""","""adwords.google.com""","""Chrome""","""Windows""",false,"""desktop""","""Americas""","""Northern America""","""United States""","""not available in demo dataset""","""not available in demo dataset""","""not available in demo dataset""",1,1,null,null,null,null


In [60]:
df = df.with_columns(pl.col('date').str.strptime(pl.Date, format="%Y%m%d"))
df.head()

date,visitId,visitNumber,visitStartTime,referralPath,source,browser,operatingSystem,isMobile,deviceCategory,continent,subContinent,country,region,metro,city,visits,pageviews,timeOnSite,transactions,transactionRevenue,totalTransactionRevenue
date,i64,i64,i64,str,str,str,str,bool,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64
2017-08-01,1501591568,1,1501591568,null,"""(direct)""","""Chrome""","""Windows""",false,"""desktop""","""Europe""","""Southern Europe""","""Greece""","""not available in demo dataset""","""not available in demo dataset""","""not available in demo dataset""",1,1,null,null,null,null
2017-08-01,1501589647,2,1501589647,"""/analytics/web/""","""analytics.google.com""","""Chrome""","""Windows""",false,"""desktop""","""Asia""","""Southern Asia""","""India""","""Maharashtra""","""(not set)""","""Mumbai""",1,1,null,null,null,null
2017-08-01,1501616621,1,1501616621,"""/analytics/web/""","""analytics.google.com""","""Chrome""","""Windows""",false,"""desktop""","""Europe""","""Northern Europe""","""United Kingdom""","""not available in demo dataset""","""not available in demo dataset""","""not available in demo dataset""",1,1,null,null,null,null
2017-08-01,1501601200,1,1501601200,"""/analytics/web/""","""analytics.google.com""","""Firefox""","""Windows""",false,"""desktop""","""Americas""","""Northern America""","""United States""","""Texas""","""Dallas-Ft. Worth TX""","""Dallas""",1,1,null,null,null,null
2017-08-01,1501615525,1,1501615525,"""/analytics/web/""","""adwords.google.com""","""Chrome""","""Windows""",false,"""desktop""","""Americas""","""Northern America""","""United States""","""not available in demo dataset""","""not available in demo dataset""","""not available in demo dataset""",1,1,null,null,null,null


### Data Pre-processing Pipeline

In [ ]:
def preprocessing_pipeline(df: pl.DataFrame) -> pl.DataFrame:
    """
    Args:
        df : pl.DataFrame

    Returns:
        pl.DataFrames: pre-processed dataframe
    """

    print("Dataframe first 5 rows")
    print(df.head())
    print(" ")
    print("Dataframe shape")
    print(df.shape)
    print(" ")
    print("Dataframe columns")
    print(df.columns)
    print(" ")
    print("Dataframe null values")
    print(df.select(pl.all().is_null().sum()))

In [ ]:
preprocessing_pipeline(category_tree_df)

Dataframe first 5 rows
shape: (5, 2)
┌────────────┬──────────┐
│ categoryid ┆ parentid │
│ ---        ┆ ---      │
│ i64        ┆ i64      │
╞════════════╪══════════╡
│ 1016       ┆ 213      │
│ 809        ┆ 169      │
│ 570        ┆ 9        │
│ 1691       ┆ 885      │
│ 536        ┆ 1691     │
└────────────┴──────────┘
 
Dataframe shape
(1669, 2)
 
Dataframe columns
['categoryid', 'parentid']
 
Dataframe null values
shape: (1, 2)
┌────────────┬──────────┐
│ categoryid ┆ parentid │
│ ---        ┆ ---      │
│ u32        ┆ u32      │
╞════════════╪══════════╡
│ 0          ┆ 25       │
└────────────┴──────────┘


In [ ]:
preprocessing_pipeline(item_df)

Dataframe first 5 rows
shape: (5, 4)
┌───────────────┬────────┬────────────┬─────────────────────────────────┐
│ timestamp     ┆ itemid ┆ property   ┆ value                           │
│ ---           ┆ ---    ┆ ---        ┆ ---                             │
│ i64           ┆ i64    ┆ str        ┆ str                             │
╞═══════════════╪════════╪════════════╪═════════════════════════════════╡
│ 1435460400000 ┆ 460429 ┆ categoryid ┆ 1338                            │
│ 1441508400000 ┆ 206783 ┆ 888        ┆ 1116713 960601 n277.200         │
│ 1439089200000 ┆ 395014 ┆ 400        ┆ n552.000 639502 n720.000 42456… │
│ 1431226800000 ┆ 59481  ┆ 790        ┆ n15360.000                      │
│ 1431831600000 ┆ 156781 ┆ 917        ┆ 828513                          │
└───────────────┴────────┴────────────┴─────────────────────────────────┘
 
Dataframe shape
(20275902, 4)
 
Dataframe columns
['timestamp', 'itemid', 'property', 'value']
 
Dataframe null values
shape: (1, 4)
┌───────────┬─

### Conclusions 
- The datasets are very clean with virtually no null values. 
- Consequently they will not require any cleaning 